<a href="https://colab.research.google.com/github/Clarice-Satiko-Aoto/b2b-analyis/blob/feat/B2B_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bibliotecas

In [2]:
#importar bibliotecas
import pandas as pd
import numpy as np

# bibliotecas para trabalhar offline com a biblioteca
import plotly
import plotly.offline as py
import plotly.express as px
import plotly.graph_objs as go # criará de fato os gráficos
from plotly.offline import plot, iplot
import cufflinks as cf # para conectar o plotly ao pandas
cf.go_offline()
plotly.offline.init_notebook_mode(connected = True)

import ipywidgets as widgets
from IPython.display import display

#warnings
import warnings
warnings.filterwarnings("ignore")

In [3]:
#Para renderizar o plotly no colab

import plotly.io as pio
# pio.renderers
pio.renderers.default = 'colab'

# Carregar arquivos

In [4]:
# descrever o que é o arquivo
df_hist = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/vendas/data/tratado/historico_completo.csv",
                      parse_dates=['data'])


In [5]:
supervisor = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/vendas/data/PBSupervisor.csv') #
vendedor = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/vendas/data/PBVendedor.csv') #

In [6]:
#criar slice da data

# Crie um slider para escolher a data inicial e final
date_range_slider = widgets.SelectionRangeSlider(
    options=pd.date_range(df_hist['data'].min(), df_hist['data'].max(), freq='D'),
    index=(0, len(pd.date_range(df_hist['data'].min(), df_hist['data'].max(), freq='D')) - 1),
    description='Período',
    layout={'width': '500px'}
)

# Função para filtrar o DataFrame com base no intervalo de datas selecionado
def update_table(date_range):
    start_date, end_date = date_range
    filtered_df = df_hist[(df_hist['data'] >= start_date) & (df_hist['data'] <= end_date)]
    # O restante do seu código para criar a tabela ou gráfico aqui

# Conecte a função ao evento de alteração do slider
widgets.interactive(update_table, date_range=date_range_slider)

# Exiba o slider
display(date_range_slider)


SelectionRangeSlider(description='Período', index=(0, 1048), layout=Layout(width='500px'), options=(Timestamp(…

# Funções

In [7]:
def mudar_tipo_de_dados(dataframe, tipos_de_dados):
    """
    Muda o tipo de dados de colunas em um DataFrame.

    Parâmetros:
    - dataframe: DataFrame Pandas
    - tipos_de_dados: Lista de pares (nome da coluna, tipo de dados)
    """

    for coluna, tipo in tipos_de_dados:
        if coluna in dataframe.columns:
            dataframe[coluna] = dataframe[coluna].astype(tipo)

# Mudar tipo de dado

In [8]:
tipos_de_dados = [
    ('gerente_id', object),
    ('categoria_id', object),
    ('supervisor_id', object),
    ('funcionario_id', object),
    ('vendedor_id', object),
    ('produtto_id', object),
    ('itens_venda_id', object),
    ('cliente_id', object),
    ('vendas_id', object),
    ('geografia_id', object)
    ]

In [9]:
mudar_tipo_de_dados(df_hist, tipos_de_dados)

# Mapear nomes supervisor, gerente e vendedor

In [10]:
supervisor_nome = dict(supervisor[['id', 'descricao']].values.tolist())
supervisor_nome

{1: 'Fernando Silva',
 2: 'Emily Rocha',
 3: 'Diego Araujo',
 4: 'Diogo Carvalho',
 5: 'Sofia Ribeiro'}

In [11]:
df_hist['supervisor_nome'] = df_hist['supervisor_id'].map(supervisor_nome)

In [12]:
gerente_nome = dict(supervisor[['gerente_id', 'gerente_descricao']].drop_duplicates().values.tolist())
gerente_nome

{1: 'Victor Castro', 2: 'Gabriel Azevedo'}

In [13]:
df_hist['gerente_nome'] = df_hist['gerente_id'].map(gerente_nome)

In [14]:
vendedor_nome = dict(vendedor[['id', 'descricao']].values.tolist())
vendedor_nome

{1: 'Augusto Bradão',
 2: 'Roberto Silva',
 3: 'Felipe Gonçalves',
 4: 'Isabella Machado',
 5: 'Estevan Souza',
 6: 'Rodrigo Santiago',
 7: 'Flávia Alessandra',
 8: 'Julio Lima',
 9: 'Mateus Costa',
 10: 'Fernanda Oliveira',
 11: 'Bruna Maria',
 12: 'Flávio Roberto'}

In [15]:
df_hist['vendedor_nome'] = df_hist['vendedor_id'].map(vendedor_nome)

In [16]:
df_hist.drop(columns=['supervisor_id', 'gerente_id', 'vendedor_id'], inplace=True)

# Criar coluna ano-mes

In [17]:
df_hist['ano_mes'] = df_hist['data'].dt.to_period('M').astype(str)

# Tira teima

In [19]:
# df_hist.query("gerente_id==1 & categoria_id ==4 & data >'2022-12-31' & data < '2023-04-01'").sort_values('data').set_index('data').resample('M')['meta_mensal'].first().sum()

# EDA

## 1) Quantas unidades foram vendidas durante o todo o período?

In [20]:
qde_vendidas = df_hist['item_quantidade'].sum()
qde_vendidas

67929.0

## 2) preço médio unitário

In [21]:
preco_medio_unit = round(df_hist['valor_unitario'].mean(),2)
preco_medio_unit

151.71

## 3) Qual faturamento líquido

In [22]:
faturamento_liquido = round((df_hist['item_quantidade'] * df_hist['valor_unitario'] - df_hist['valor_desconto']).sum(),2)
faturamento_liquido

8611082.84

## 4) ticket medio

In [23]:
nfe_unicos = df_hist['nfe'].nunique()

ticket_medio = round(faturamento_liquido/nfe_unicos,2)
ticket_medio

2509.79

## 5) evolução das vendas

In [24]:
#Preparar os dados
vendas_mes = (
    df_hist.groupby(by='ano_mes')['valor_venda'].sum())

# destacar uma barra
cores=[]

# valores
media = vendas_mes.values.mean()

for x in vendas_mes.values:
  if x < media:
    cores.append('#FF3F3D')
  else:
    cores.append('#3BBF61')


#gráfico de barras
data = [
    go.Bar(
        x=vendas_mes.index,
        y=vendas_mes.values,
        marker={'color': cores,
                # 'line': {'color': '#805E69',
                        #  'width': 1},
                },
        opacity=.9
    )
]

#criar layout
config_layout = go.Layout(title='Vendas mensal',
                          yaxis={'title': 'Vendas em R$'},
                          # xaxis={'title': 'Período'}
                          )

#objeto figura
fig = go.Figure(data=data, layout=config_layout)

#plotar o gráfico
py.iplot(fig)

In [25]:
#Preparar os dados
vendas_mes = (
    df_hist.groupby(by='ano_mes')['valor_venda'].sum())

# destacar uma barra
cores=[]

# valores
media = vendas_mes.values.mean()
max_vendas = vendas_mes.values.max()

for x,y in zip(vendas_mes.values, vendas_mes.index):
    # if x < media:
    #   cores.append('#FF3F3D')
    # else:
    #   cores.append('#3BBF61')

    if x == max_vendas:
      mes_max_vendas = y
      cores.append('#1B1A8A')
    else:
      cores.append('#6689B2')



#gráfico de barras
data = [
    go.Bar(
        x=vendas_mes.index,
        y=vendas_mes.values,
        marker={'color': cores,
                # 'line': {'color': '#805E69',
                        #  'width': 1},
                },
        opacity=.9
    )
]

#criar layout
config_layout = go.Layout(title='Vendas mensal',
                          annotations=[
                              {'text': 'Mês destaque',
                               'x': mes_max_vendas,
                               'y': max_vendas}
                          ],
                          yaxis={'title': 'Vendas em R$'},
                          # xaxis={'title': 'Período'}
                          )

#objeto figura
fig = go.Figure(data=data, layout=config_layout)

#plotar o gráfico
py.iplot(fig)

## 6) O que vendeu mais em cada ano

In [26]:
vendas_anual = (df_hist
                .groupby([df_hist['data'].dt.to_period('Y'),'categoria_descricao'])['categoria_id'].count()
                .reset_index()
                .sort_values(['data', 'categoria_id'], ascending=True) )

vendas_anual['data'] = vendas_anual['data'].astype(str)
vendas_anual

,data,categoria_descricao,categoria_id
1,2021,Camiseta,1183
0,2021,Blazer,1285
3,2021,Manga Longa,5153
2,2021,Manga Curta,8144
5,2022,Camiseta,808
4,2022,Blazer,857
7,2022,Manga Longa,3673
6,2022,Manga Curta,6648
9,2023,Camiseta,892
8,2023,Blazer,976


In [27]:
# Crie o gráfico de barras
cor_ano=['#3BBF61', '#6689B2', '#1B1A8A']
fig = px.bar(vendas_anual, x='categoria_descricao', y='categoria_id', color='data',
             labels={'categoria_id': 'Quantidade', 'data': 'Ano'},
             title='Comparação de Quantidade por Categoria e Ano',
             barmode='group',
             color_discrete_sequence=cor_ano,
             )

# Posicione a legenda no canto superior esquerdo
fig.update_layout(legend=dict(x=0, y=1.09, traceorder='normal', orientation='h', ))

# Exiba o gráfico
py.iplot(fig)

In [28]:
vendas_por_categoria = df_hist.groupby(by='categoria_descricao')['categoria_id'].count().sort_values(ascending=False)
vendas_por_categoria

categoria_descricao
Manga Curta    21158
Manga Longa    12801
Blazer          3118
Camiseta        2883
Name: categoria_id, dtype: int64

In [29]:
#criar o gráfico
categoria = go.Bar(
    y=vendas_por_categoria.values,
    x=vendas_por_categoria.index,
    marker={'color':'#1B1A8A'},
    orientation='v'
)

#armazenar o gráfico em uma lista
data = [categoria]

#criar layout
config_layout = go.Layout(title='Vendas por categoria de produtos',
                          xaxis=dict(
                              titlefont=dict(
                                  size=36,
                                  color='#1B1A8A'
                              ),
                          # tickangle=75

                          ),
                          yaxis={'title':'Qde vendida',
                                 'range': [0, max(vendas_por_categoria.values)*1.1]
                                 })

#objeto figura
fig = go.Figure(data=data, layout=config_layout)

#plotar o gráfico
py.iplot(fig)

### Verificando as vendas da categoria Manga curta

In [30]:
vendas_anual = (
    df_hist
    .groupby([df_hist['data'].dt.to_period('M'), 'categoria_descricao', 'produto_descricao'])
    .agg(
        quantidade=('item_quantidade', 'sum'),
        valor=('valor_venda', 'sum')
        )
    .reset_index()
    ).sort_values(['data', 'valor'])


mangas_curtas = vendas_anual.query("categoria_descricao =='Manga Curta' ").groupby(['data', 'produto_descricao']).agg(quantidade_vendida=('quantidade', 'sum'), faturamento_medio=('valor', 'mean')).reset_index()
mangas_curtas['data'] = mangas_curtas['data'].astype(str)


In [31]:
# Crie o gráfico de linhas
fig = px.line(mangas_curtas,
              x='data',
              y='faturamento_medio',
              color='produto_descricao',
              labels={'faturamento_medio': 'Faturamento Médio',
                      'data': 'Mês',
                      'produto_descricao': 'produto'},
              title='Evolução Mensal do Faturamento por Produto')
#plotar o gráfico
py.iplot(fig)

## 7) Relaçao entre o valor do desconto e o valor final de venda

In [32]:
#Relaçao entre o valor do desconto e o valor final de venda
#criar o gráfico
valor = go.Scatter(
    x=df_hist['valor_liquido'],
    y=df_hist['valor_venda'],
    mode='markers',
    marker={'color':'#1B1A8A'}
)

#armazenar em lista
data=[valor]

#criar layout
config_layout = go.Layout(title='Valor desconto x valor venda',
                          xaxis={'title':'Valor do desconto'},
                          yaxis={'title':'Valor de venda'}
                          )

#objeto figura
fig = go.Figure(data=data, layout=config_layout)

#plotar o gráfico
py.iplot(fig)

## 8) Clientes

In [33]:
# Encontrar o cliente com a maior venda em cada ano
maiores_clientes = df_hist.groupby(df_hist['data'].dt.to_period('Y'))['valor_venda'].idxmax()

# Exibir os maiores clientes por ano
for idx in maiores_clientes:
    ano = df_hist.loc[idx, 'data']
    cliente = df_hist.loc[idx, 'cliente']
    venda = df_hist.loc[idx, 'valor_venda']
    print(f"No ano {ano}, o maior cliente foi {cliente} com uma venda de R${venda:.2f}.")

No ano 2021-08-11 00:00:00, o maior cliente foi Kemmer-Tillman com uma venda de R$10478.10.
No ano 2022-09-21 00:00:00, o maior cliente foi Boehm-Gaylord com uma venda de R$11493.55.
No ano 2023-04-11 00:00:00, o maior cliente foi Gerhold-Cole com uma venda de R$10564.50.


### 8.1) Explorar mais sobre os clientes.
ex) localização, comportamento de compras, a qual supervisor está ligado, etc

## 9) metas

In [34]:
#Agrupar os dados para verificar se a equipe atingiu a meta mensal por categoria
vendas_equipe_mensal = (
    df_hist
    .groupby(['gerente_nome',
              'categoria_descricao',
              df_hist['data'].dt.to_period('M') ])
    .agg(
        meta_mensal_equipe=('meta_mensal', 'first'),
        venda_mensal = ('valor_venda', 'sum'))
    .reset_index()

)
vendas_equipe_mensal['data'] = vendas_equipe_mensal['data'].astype(str)
# vendas_equipe_mensal['data'] = vendas_equipe_mensal['data'].dt.to_timestamp(freq='M')
# vendas_equipe_mensal['data'] = pd.to_datetime(vendas_equipe_mensal['data'])
vendas_equipe_mensal.tail()

,gerente_nome,categoria_descricao,data,meta_mensal_equipe,venda_mensal
259,Victor Castro,Manga Longa,2023-06,40615.384615,711202.15
260,Victor Castro,Manga Longa,2023-07,31984.615385,740289.95
261,Victor Castro,Manga Longa,2023-08,35030.769231,1087983.65
262,Victor Castro,Manga Longa,2023-09,31984.615385,950646.05
263,Victor Castro,Manga Longa,2023-10,17261.538462,909410.60


In [35]:


# Converta a coluna "data" para o tipo de dado adequado (Timestamp)
vendas_equipe_mensal['data'] = pd.to_datetime(vendas_equipe_mensal['data'])


# Crie um gráfico de linha para representar as metas mensais e vendas mensais
fig_linha = px.line(
    vendas_equipe_mensal,
    x='data',
    y=[ 'venda_mensal'], #'meta_mensal_equipe',
    color='gerente_nome',
    facet_row='categoria_descricao',
    labels={'value': 'Valor', 'variable': 'Categoria'},
    title='Vendas Mensais por Categoria',
    height=600
)



# Configure a legenda para o gráfico de linha
fig_linha.update_layout(
    legend=dict(x=0, y=1.1, traceorder='normal', orientation='h'),
    xaxis={'title': None, 'side':'right'},
    yaxis={'title': None}

)

# Exiba os gráficos

fig_linha.show()


In [67]:
#Plotar os graficos lado a lado
# Crie um gráfico de linha para representar as metas mensais e vendas mensais
fig_linha = px.line(
    vendas_equipe_mensal,
    x='data',
    y=['venda_mensal'], #'meta_mensal_equipe',
    color='gerente_nome',
    facet_col='categoria_descricao',  # Use facet_col em vez de facet_row
    labels={'value': 'Valor', 'variable': 'Categoria'},
    title='Vendas Mensais por Categoria',
    height=600,

)

# Configure a legenda para o gráfico de linha
fig_linha.update_layout(
    legend=dict(x=0, y=1.1, traceorder='normal', orientation='h'),
)

# Exiba os gráficos
fig_linha.show()


In [52]:
fig = px.sunburst(vendas_equipe_mensal,
                  path=['gerente_nome', 'categoria_descricao'],
                  values='venda_mensal',
                  color='venda_mensal',
                  hover_data=['venda_mensal'],
                  color_continuous_scale='RdBu')

# Configure a legenda para o gráfico de linha
fig.update_layout(
    legend=dict(x=0, y=1.1, traceorder='normal', orientation='h'),
)
fig.show()

In [103]:
#Agrupar os dados para verificar se o vendedor bateu a meta mensal ou não por categoria
vendas_individual_mensal = (
    df_hist.query("data > '2021-12-31' & data < '2023-10-23'")
    .groupby(['supervisor_nome',
              'gerente_nome',
              'vendedor_nome',
              'categoria_descricao',
              df_hist['data'].dt.to_period('M') ])
    .agg(
        meta_mensal_indiv=('meta_mensal_indiv', 'first'),
        venda_mensal = ('valor_venda', 'sum'))
    .reset_index()

)
vendas_individual_mensal['data'] = vendas_individual_mensal['data'].astype(str)
# vendas_individual_mensal['data'] = vendas_individual_mensal['data'].dt.to_timestamp(freq='M')
# vendas_individual_mensal['data'] = pd.to_datetime(vendas_individual_mensal['data'])

#Verificar se o vendedor atingiu a meta mensal
vendas_individual_mensal['atingiu_meta'] = vendas_individual_mensal['venda_mensal'] >= vendas_individual_mensal['meta_mensal_indiv']

vendas_individual_mensal.tail()

,supervisor_nome,gerente_nome,vendedor_nome,categoria_descricao,data,meta_mensal_indiv,venda_mensal
1034,Sofia Ribeiro,Gabriel Azevedo,Mateus Costa,Manga Longa,2023-06,9589.74,269360.90
1035,Sofia Ribeiro,Gabriel Azevedo,Mateus Costa,Manga Longa,2023-07,6865.38,55548.80
1036,Sofia Ribeiro,Gabriel Azevedo,Mateus Costa,Manga Longa,2023-08,7519.23,44956.70
1037,Sofia Ribeiro,Gabriel Azevedo,Mateus Costa,Manga Longa,2023-09,6865.38,208206.85
1038,Sofia Ribeiro,Gabriel Azevedo,Mateus Costa,Manga Longa,2023-10,4794.87,311446.95


In [116]:
resumo_vendedores = (
    vendas_individual_mensal
    .groupby('vendedor_nome')
    .agg(
        media_meta=pd.NamedAgg(column='meta_mensal_indiv', aggfunc='mean'),
        total_vendas=pd.NamedAgg(column='venda_mensal', aggfunc='sum'),
        metas_atingidas=pd.NamedAgg(column='atingiu_meta', aggfunc='sum'),
    )
    .reset_index()
    .sort_values('metas_atingidas',
                 ascending=False,
                 ignore_index=True)
    )

resumo_vendedores

,vendedor_nome,media_meta,total_vendas,metas_atingidas
0,Isabella Machado,5065.152727,8780605.30,87
1,Roberto Silva,5086.225057,12179891.55,86
2,Bruna Maria,5130.130706,12183488.80,85
3,Julio Lima,5022.778736,8894184.10,84
4,Mateus Costa,5004.437586,9564190.20,84
5,Estevan Souza,5186.222471,9747842.05,83
6,Flávia Alessandra,5122.544186,8828945.75,83
7,Rodrigo Santiago,5188.661905,8996644.05,83
8,Fernanda Oliveira,5120.243256,8083652.65,82
9,Augusto Bradão,5160.199880,10078077.90,78


In [129]:

# Supondo também que a coluna 'atingiu_meta' seja booleana e precisa ser convertida para string
vendas_individual_mensal['atingiu_meta'] = vendas_individual_mensal['atingiu_meta'].astype(str)

# Criar gráfico de barras empilhadas
fig = px.bar(vendas_individual_mensal,
             x='data',
             y='venda_mensal',  # Pode ser outra métrica, dependendo do que você deseja analisar
             color='atingiu_meta',  # Cores diferentes para quem atingiu ou não a meta
            #  facet_row='vendedor_nome',
             facet_col='vendedor_nome',  # Facetar por vendedor
             labels={'venda_mensal': 'Valor de Vendas', 'atingiu_meta': 'Atingiu a Meta'},
             title='Desempenho Mensal em Relação à Meta por Vendedor',
             barmode='stack'  # Barras empilhadas
            )

# Atualizar layout
fig.update_layout(
    xaxis_title='Data',
    yaxis_title='Valor de Vendas',
    legend=dict(x=0, y=1.1, traceorder='normal', orientation='h'),
    showlegend=True,

)

# Exibir gráfico
fig.show()


In [84]:
# Localizar os valores NaN
valores_nan = df_hist.isna()

# filtrar as linhas com NaN
linhas_com_nan = df_hist[valores_nan.any(axis=1)]
linhas_com_nan.head()

,vendas_id,data,nfe,cliente_id,nf_desconto,valor_liquido,valor_venda,itens_venda_id,produto_id,item_quantidade,...,categoria_descricao,meta_mensal,qde_funcionarios,meta_mensal_indiv,dias_uteis,meta_diaria_indiv,supervisor_nome,gerente_nome,vendedor_nome,ano_mes
0,1.0,2021-02-16,SO43660,614.0,24.90,265.0,240.10,13.0,43.0,1.0,...,Manga Curta,NaN,NaN,NaN,NaN,NaN,Emily Rocha,Victor Castro,Isabella Machado,2021-02
2,993.0,2021-06-24,SO46324,259.0,62.80,1014.0,951.20,7650.0,43.0,1.0,...,Manga Curta,NaN,NaN,NaN,NaN,NaN,Sofia Ribeiro,Gabriel Azevedo,Fernanda Oliveira,2021-06
3,7.0,2021-02-17,SO43665,347.0,112.95,1711.0,1598.05,61.0,43.0,1.0,...,Manga Curta,NaN,NaN,NaN,NaN,NaN,Sofia Ribeiro,Gabriel Azevedo,Felipe Gonçalves,2021-02
5,12.0,2021-02-18,SO43670,266.0,26.35,602.0,575.65,112.0,43.0,1.0,...,Manga Curta,NaN,NaN,NaN,NaN,NaN,Sofia Ribeiro,Gabriel Azevedo,Fernanda Oliveira,2021-02
7,454.0,2021-04-26,SO44762,202.0,306.85,3776.0,3469.15,3627.0,43.0,1.0,...,Manga Curta,NaN,NaN,NaN,NaN,NaN,Emily Rocha,Victor Castro,Bruna Maria,2021-04
